# Example of Updating a Dataset

The [dataset](https://github.com/open-reaction-database/ord-data/blob/2a6dff66c02f5d1c9b1589691f845b941c981a7a/data/a1/ord_dataset-a12fa15d036d489c971b0b514caeae52.pb.gz) was found to have some incorrect SMILES strings after it had been published in [ord-data](https://github.com/open-reaction-database/ord-data). This notebook details the workflow used to update the dataset whilst maintaining the dataset and reaction ids.

In [1]:
# Import modules
import ord_schema
from ord_schema import message_helpers, validations, updates
from ord_schema.proto import dataset_pb2
from ord_schema.proto import reaction_pb2

import math
import pandas as pd
import numpy as np
import os
#import wget
from datetime import datetime

from glob import glob

## Load in the dataset to be updated

In [2]:
# Download the dataset
# on Linux
#url = "https://github.com/open-reaction-database/ord-data/blob/main/data/a1/ord_dataset-a12fa15d036d489c971b0b514caeae52.pb.gz"
#pb = wget.download(url)

# on Windows - manually download the dataset to a directory containing this notebook


In [3]:
# Load the dataset to be updated
pb = './ord_dataset-a12fa15d036d489c971b0b514caeae52_old.pb.gz'
data = message_helpers.load_message(pb, dataset_pb2.Dataset)

In [4]:
# Inspect the dataset
print(f"Dataset name is {data.name}")
print(f"Dataset descripton is {data.description}")

Dataset name is Fig 7 data from Golden paper
Dataset descripton is Cross-coupling of 3-(1-chloroethyl)pyridine with a range of  nucleophiles. Data from Fig 7 of https://doi.org/10.1016/j.chempr.2024.04.001.


In [5]:
# To find the messages which need updating it can be helpful to view the dataset as a DataFrame
# Convert dataset to pandas dataframe
df = message_helpers.messages_to_dataframe(data.reactions, drop_constant_columns=True)

# View dataframe
df

,identifiers[1].value,"inputs[""Nucleophile in DMF""].components[0].identifiers[0].value",outcomes[0].products[0].identifiers[0].value,outcomes[0].products[0].measurements[0].percentage.value,reaction_id
0,A1,NCCSCc1ccco1,CC(NCCSCc1ccco1)c1cccnc1,61.0,ord-eafdcf86ab4049bdaced8adc28ff4b42
1,A2,CC(C)(C)Oc1ccc(CCCN)cc1,CC(NCCCc1ccc(OC(C)(C)C)cc1)c1cccnc1,67.0,ord-ab68ccc98d184f9c82b51e02e72399eb
2,A3,Cc1ccc(S(=O)(=O)CCN)cc1,CC(N[C@H]1CCCC[C@@H]1OCc1ccccc1)c1cccnc1,56.0,ord-85c8f00118eb4899b36ddc53112d676f
3,A4,NCC1(N2CCCCC2)CCOCC1,Cc1ccc(S(=O)(=O)CCNC(C)c2cccnc2)cc1,63.0,ord-0e5399b5617a489e98405c3172c8779d
4,A5,N[C@@H]1C[C@H]2CC[C@@H]1C2,CC(NCC1(N2CCCCC2)CCOCC1)c1cccnc1,63.0,ord-ed93dbd8dace4e078573612131d2ef2f
...,...,...,...,...,...
283,H8,Cc1ccccc1-n1ncc(C(=O)O)c1C,Cc1ccccc1-n1ncc(C(=O)OC(C)c2cccnc2)c1C,81.0,ord-19f1549ea6f44eae96973699b7e6d0a0
284,H9,O=C(O)c1cn(Cc2ccccc2)nn1,CC(OC(=O)c1cn(Cc2ccccc2)nn1)c1cccnc1,91.0,ord-0133a2a17eb34832a8f573384e89d644
285,H10,O=C(O)c1nn(CC2CC2)c2ccccc12,CC(OC(=O)c1nn(CC2CC2)c2ccccc12)c1cccnc1,71.0,ord-c30c707842604783b1da44ea48737140
286,H11,O=C(O)c1ncn2c1CCCC2,CC(OC(=O)c1ncn2c1CCCC2)c1cccnc1,96.0,ord-79706a33f8734f4088c293968ee9782f


### Notes
The column names can help identify the messages which may be updated.

## Load in the new source data

In [6]:
# Load in the replacement data
path = './Fig_7_data_corrected.csv'
df_replace = pd.read_csv(path)
df_replace

,Well,nucleophile_smiles,product_smiles,LCAP
0,A1,NCCSCC1=CC=CO1,CC(C1=CC=CN=C1)NCCSCC2=CC=CO2,61
1,A2,NCCCC2=CC=C(C=C2)OC(C)(C)C,CC(C3=CC=CN=C3)NCCCC4=CC=C(C=C4)OC(C)(C)C,67
2,A3,NCCS(=O)(C3=CC=C(C)C=C3)=O,O=S(CCNC(C)C5=CC=CN=C5)(C6=CC=C(C)C=C6)=O,56
3,A4,NCC4(CCOCC4)N5CCCCC5,CC(C7=CC=CN=C7)NCC8(CCOCC8)N9CCCCC9,63
4,A5,N[C@@H]6C[C@H]7CC[C@@H]6C7,CC(C%10=CC=CN=C%10)N[C@@H]%11C[C@H]%12CC[C@@H]...,63
...,...,...,...,...
283,H8,Cc1ccccc1-n1ncc(C(=O)O)c1C,Cc1ccccc1-n1ncc(C(=O)OC(C)c2cccnc2)c1C,81
284,H9,O=C(O)c1cn(Cc2ccccc2)nn1,CC(OC(=O)c1cn(Cc2ccccc2)nn1)c1cccnc1,91
285,H10,O=C(O)c1nn(CC2CC2)c2ccccc12,CC(OC(=O)c1nn(CC2CC2)c2ccccc12)c1cccnc1,71
286,H11,O=C(O)c1ncn2c1CCCC2,CC(OC(=O)c1ncn2c1CCCC2)c1cccnc1,96


### Notes
We know that some, but not all, of the nucleophile_smiles and product_smiles have been corrected. Well, and LCAP are unchanged.

## Update the dataset

In [8]:
# Make a new copy of the dataset for updating
data2 = message_helpers.load_message(pb, dataset_pb2.Dataset)

In [9]:
# Iterate over the dataset and update reactions
for index, row in df_replace.iterrows():
    # check if the reaction needs to be updated
    if  data.reactions[index].inputs["Nucleophile in DMF"].components[0].identifiers[0].value != row['nucleophile_smiles'] or \
        data.reactions[index].outcomes[0].products[0].identifiers[0].value != row['product_smiles']:

        data2.reactions[index].inputs["Nucleophile in DMF"].components[0].identifiers[0].value = row['nucleophile_smiles']
        data2.reactions[index].outcomes[0].products[0].identifiers[0].value = row['product_smiles']
        # Add a record_modified event.
        event = data2.reactions[index].provenance.record_modified.add()
        event.time.value = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
        event.person.CopyFrom(
            reaction_pb2.Person(
                name="Benjamin J. Deadman", organization="Open Reaction Database", orcid="0000-0001-8463-8199", email="ben@bjdeadman.co.uk"
            )
        )
        event.details = "Corrected the reactant and product SMILES strings"
        #print(f"reaction {index} was updated")
    
    else:    
        #print(f"reaction {index} is unchaged")
        pass

In [10]:
# Inspect a single reaction to view the updated fields in the reaction record
r_index = 1 # check the reaction at this index
print(data2.reactions[r_index].inputs["Nucleophile in DMF"].components[0].identifiers[0])
print(data2.reactions[r_index].outcomes[0].products[0].identifiers[0])
print(data2.reactions[r_index].provenance)

type: SMILES
value: "NCCCC2=CC=C(C=C2)OC(C)(C)C"

type: SMILES
value: "CC(C3=CC=CN=C3)NCCCC4=CC=C(C=C4)OC(C)(C)C"

doi: "10.1016/j.chempr.2024.04.001"
publication_url: "https://doi.org/10.1016/j.chempr.2024.04.001"
record_created {
  time {
    value: "7/24/2024, 3:36:42 PM"
  }
  person {
    name: "Spencer P. Heins"
    organization: "UW-Madison"
    email: "sheins@wisc.edu"
  }
}
record_modified {
  time {
    value: "Wed Jul 31 22:03:49 2024"
  }
  person {
    username: "github-actions"
    email: "github-actions@github.com"
  }
  details: "Automatic updates from the submission pipeline."
}
record_modified {
  time {
    value: "12/18/2024, 16:17:40"
  }
  person {
    name: "Benjamin J. Deadman"
    orcid: "0000-0001-8463-8199"
    organization: "Open Reaction Database"
    email: "ben@bjdeadman.co.uk"
  }
  details: "Corrected the reactant and product SMILES strings"
}



## Comparing the datasets

In [14]:
# Convert the datasets to DataFrames, and keep constant columns so that we can see any added messages
df1 = message_helpers.messages_to_dataframe(data.reactions, drop_constant_columns=False)
df2 = message_helpers.messages_to_dataframe(data2.reactions, drop_constant_columns=False)

In [18]:
# Compare the shape of the DataFrames to confirm that the updated df2 has additional columns for the record_modified metadata
print(df1.shape)
print(df2.shape)

(288, 127)
(288, 133)


In [26]:
# Convert the datasets to compact DataFrames
df1b = message_helpers.messages_to_dataframe(data.reactions, drop_constant_columns=True)
df2b = message_helpers.messages_to_dataframe(data2.reactions, drop_constant_columns=True)

In [28]:
# Inspect the DataFrame info, the non-null count in record_modified column tells us how many reaction records were updated
print(df1b.info())
print(df2b.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 5 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   identifiers[1].value                                             288 non-null    object 
 1   inputs["Nucleophile in DMF"].components[0].identifiers[0].value  288 non-null    object 
 2   outcomes[0].products[0].identifiers[0].value                     288 non-null    object 
 3   outcomes[0].products[0].measurements[0].percentage.value         288 non-null    float64
 4   reaction_id                                                      288 non-null    object 
dtypes: float64(1), object(4)
memory usage: 11.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 11 columns):
 #   Column                                                           Non-N

In [36]:
# Compare the DataFrames
# Select only those columns which are in both DataFrames
cols_list = ['identifiers[1].value', 
             'inputs["Nucleophile in DMF"].components[0].identifiers[0].value', 
             'outcomes[0].products[0].identifiers[0].value', 
             'outcomes[0].products[0].measurements[0].percentage.value', 
             'reaction_id'
            ]
# Use Pandas df.compare to highlight the differences in row format
df1b[cols_list].compare(df2b[cols_list], align_axis=0, result_names=('old', 'new'))

inputs["Nucleophile in DMF"].components[0].identifiers[0].value  \
0   old                                       NCCSCc1ccco1                
    new                                     NCCSCC1=CC=CO1                
1   old                            CC(C)(C)Oc1ccc(CCCN)cc1                
    new                         NCCCC2=CC=C(C=C2)OC(C)(C)C                
2   old                            Cc1ccc(S(=O)(=O)CCN)cc1                
...                                                    ...                
188 new                       OCC1=CN(C2=CC=C(Cl)C=C2)N=N1                
189 old                              OCCC1CCN(Cc2ccccc2)C1                
    new                           OCCC1CCN(CC2=CC=CC=C2)C1                
196 old                     CN(C)Cc1c[nH]c2cc(C(=O)O)ccc12                
    new                    OC(C1=CC(NC=C2CN(C)C)=C2C=C1)=O                

              outcomes[0].products[0].identifiers[0].value  
0   old                           CC(NCCSCc1ccco1)c1cccnc1  
    new                      CC(C1=CC=CN=C1)NCCSCC2=CC=CO2  
1   old                CC(NCCCc1ccc(OC(C)(C)C)cc1)c1cccnc1  
    new          CC(C3=CC=CN=C3)NCCCC4=CC=C(C=C4)OC(C)(C)C  
2   old           CC(N[C@H]1CCCC[C@@H]1OCc1ccccc1)c1cccnc1  
...                                                    ...  
188 new  ClC%23=CC=C(C=C%23)N%24N=NC(COC(C)C%25=CC=CN=C...  
189 old               CC(OCc1cn(-c2ccc(Cl)cc2)nn1)c1cccnc1  
    new  CC(C%26=CC=CN=C%26)OCCC%27CCN(CC%28=CC=CC=C%28...  
196 old         CC(OC(=O)c1ccc2c(CN(C)C)c[nH]c2c1)c1cccnc1  
    new     O=C(C1=CC(NC=C2CN(C)C)=C2C=C1)OC(C)C3=CC=CN=C3  

[194 rows x 2 columns]

### Reviewing the changes

- df.info() shows that new columns for record_modified meta data have been added
- 97 non-null count in these columns indicates that 97 reactions were updated
- df.compare() shows that only the reactant and the product SMILES have been modified (in 194 rows / 2 = 97 reactions). Importantly the reaction_id is unchanged.

## Preparing the dataset for upload to ord-data
We need to save the updated dataset to a compressed .pb file with the same id

In [39]:
# For the purpose of this example the original file had a '_old.pb.gz' suffix. This is removed here to create the target file path
pb_new_path = './ord_dataset-a12fa15d036d489c971b0b514caeae52.pb.gz'

In [40]:
message_helpers.write_message(data2, pb_new_path)

From here we use the normal pull request process to replace the existing dataset file in ord-data.